

Install dependencies




In [1]:
!pip3 install tensorflow==2.2 keras simpleitk

In [2]:
# !pip install git+https://github.com/JihongJu/keras-resnet3d.git

Import Packages

In [3]:
import os
# os.environ["CUDA_DEVICE_ORDER"] = "PCI_BUS_ID"   #if like me you do not have a lot of memory in your GPU
# os.environ["CUDA_VISIBLE_DEVICES"] = "" #then these two lines force keras to use your CPU
# import keras
from tensorflow import keras
import tensorflow as tf
from tensorflow.keras.models import Sequential
import matplotlib.pyplot as plt
# from keras.layers import Dense, Dropout, Activation, Flatten, Lambda
# from keras.layers import Conv2D, MaxPooling2D
# from keras.layers.normalization import BatchNormalization as BN
# from keras.layers import GaussianNoise as GN
from tensorflow.keras.layers import Dense, Flatten, Conv3D, MaxPooling3D, MaxPool3D, GlobalAveragePooling3D, Dropout, BatchNormalization
from tensorflow.keras.optimizers import Adam, SGD, RMSprop
from tensorflow.keras.utils import to_categorical
# from tensorflow.keras.utils import Sequence
from tensorflow.python.keras.utils import data_utils
from tensorflow.keras.utils import plot_model
import numpy as np
import matplotlib.pyplot as plt
import h5py
from skimage import data
from skimage.util import montage 
import skimage.transform as skTrans
from skimage.transform import rotate
from skimage.transform import resize
#from skimage.transform import resiz
from sklearn import preprocessing

from glob import glob

import SimpleITK as sitk
import nibabel as nib

**Load and prepare data**

In [4]:
from google.colab import drive
drive.mount('/content/drive')

project_dir = "drive/MyDrive/Colab Notebooks/TFM/DATA/"

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [5]:
batch_size = 1
epochs = 100
# frozen_epochs = 100
num_classes = 3
images_shape = (192,192,160)
n_channels = 1

In [6]:
# # img_route = project_dir  + 'AD/002_S_0619_I48617/002_S_0619_I48617_masked.nii.gz'
# # img_route = project_dir  + 'AD/002_S_0619_I48617/002_S_0619_I48617_masked_basic.nii.gz'
# img_route = project_dir  + 'Train/AD/023_S_0139_I31300_masked_basic.nii.gz'
# img_route = project_dir  + 'Train/MCI/067_S_0098_I65694_masked_basic.nii.gz'

# # load in sitk format (SimpleITK)
# sitk_image = sitk.ReadImage(img_route)
# # transform into a numpy array
# img = sitk.GetArrayFromImage(sitk_image)
# print(np.expand_dims(img, axis=3).shape)
# img = np.expand_dims(img, axis=3)
# sample_shape = img.shape
# # img = img / np.amax(img)
# plt.imshow(img[:,:,100, 0], cmap='gray' )
# plt.show()
# print(img[60,:,100,:])
# print(np.amax(img))

In [7]:
# # Skip 50:-50 slices since there is not much to see
# fig, ax1 = plt.subplots(1, 1, figsize = (15,15))
# ax1.imshow(rotate(montage(img[24:-23,:,:,0]), -90, resize=True), cmap ='gray')

In [8]:
le = preprocessing.LabelEncoder()
le.fit(["AD", "CN", "MCI"])

LabelEncoder()

In [9]:
    # def __get_index(data_path):
    #   files = [os.path.relpath(file_dir, data_path) for x in os.walk(data_path) for file_dir in glob(os.path.join(x[0], '*.nii.gz'))]
    #   x_index = []
    #   y_labels = []

    #   for f in files:
    #     f = f.split('/')
    #     x_index.append(f[1])
    #     y_labels.append(f[0])

    #   y_labels = le.transform(y_labels)
    #   # y_labels = to_categorical(y_labels, num_classes)

    #   print(le.inverse_transform([y_labels[1]])[0])

    #   return x_index, y_labels
    
    # a,b = __get_index(project_dir + '/Train/')
    # print(type(a))
    # print(type(b))


Custom Image Generator

In [10]:
class DataGenerator(data_utils.Sequence):
    'Generates data for Keras'
    def __init__(self, data_path, dim=(192,192,160), batch_size = 1, n_channels = 1, shuffle=True):
        'Initialization'
        self.data_path = data_path
        self.dim = dim
        self.batch_size = batch_size
        self.n_channels = n_channels
        self.shuffle = shuffle

        self.list_IDs, self.Y_labels = self.__get_index(data_path)
        self.on_epoch_end()
    
    def __get_index(self, data_path):
        files = [os.path.relpath(file_dir, data_path) for x in os.walk(data_path) for file_dir in glob(os.path.join(x[0], '*.nii.gz'))]
        x_index = []
        y_labels = []

        for f in files:
            f = f.split('/')
            x_index.append(f[1])
            y_labels.append(f[0])

        y_labels = le.transform(y_labels)
        # y_labels = to_categorical(y_labels, num_classes)

        return x_index, y_labels


    def __len__(self):
        'Denotes the number of batches per epoch'
        return int(np.floor(len(self.list_IDs) / self.batch_size))

    def __getitem__(self, index): #Index denotes the current batch on an epoch
        'Generate one batch of data'
        # Generate indexes of the batch
        indexes = self.indexes[index*self.batch_size:(index+1)*self.batch_size]

        # Find list of IDs
        Batch_ids = [self.list_IDs[k] for k in indexes]
        Batch_Y = [self.Y_labels[k] for k in indexes]
        
        # Generate data
        X = self.__data_generation(Batch_ids, Batch_Y)

        Batch_Y = to_categorical(Batch_Y, num_classes)

        return X, Batch_Y

    def on_epoch_end(self):
        'Updates indexes after each epoch'
        self.indexes = np.arange(len(self.list_IDs))
        if self.shuffle == True:
            np.random.shuffle(self.indexes)

    def __data_generation(self, Batch_ids, Batch_Y):
        'Generates data containing batch_size samples'
        # Initialization
        X = np.zeros((self.batch_size, *self.dim, self.n_channels))

        # Generate data
        for c, i in enumerate(Batch_ids): #count, element
            case_path = os.path.join(self.data_path, le.inverse_transform([Batch_Y[c]])[0])
            img_path = os.path.join(case_path, i);

            # load nibabel Method
            img = nib.load(img_path).get_fdata()    

            # # One more dimension for the channels
            img = np.expand_dims(img, axis=3)

            X[c,:,:,:,:] = resize(img, (self.dim[0], self.dim[1], self.dim[2], 1))
        return X/np.max(X) #We normalize between 0 an 1

In [11]:
# import time

# X = ['023_S_0139_I31300_masked_basic.nii.gz',
#   '023_S_0139_I52066_masked_basic.nii.gz',
#   '023_S_0139_I81883_masked_basic.nii.gz',
#   '023_S_1262_I103322_masked_basic.nii.gz',
#   '023_S_1262_I62433_masked_basic.nii.gz',
#   '023_S_1262_I76649_masked_basic.nii.gz',
#   '057_S_1373_I80258_masked_basic.nii.gz',
#   '141_S_1152_I102067_masked_basic.nii.gz',
#   '141_S_1152_I48590_masked_basic.nii.gz',
#   '141_S_1152_I79410_masked_basic.nii.gz']
# Y = [0, 0, 0, 0, 0, 0, 0, 0, 0, 0]

# data_path = project_dir + '/Train/'
# dim = (192,192,160)
# # batch_size = 10
# # n_channels = 1

# # def __data_generation(Batch_ids, Batch_Y):
# #         'Generates data containing batch_size samples' # X : (n_samples, *dim, n_channels)
# #         # Initialization
# #         X = np.zeros((batch_size, *dim, n_channels))

# #         # Generate data
# #         for c, i in enumerate(Batch_ids): #count, element
# #             case_path = os.path.join(data_path, le.inverse_transform([Batch_Y[c]])[0])
# #             img_path = os.path.join(case_path, i);

# #             # load nibabel Method
# #             img = nib.load(img_path).get_fdata()    

# #             # # load in sitk format (SimpleITK Method)
# #             # sitk_image = sitk.ReadImage(img_path)
# #             # # transform into a numpy array
# #             # img = sitk.GetArrayFromImage(sitk_image)

# #             # img = np.swapaxes(img,0,2)
# #             # img = np.swapaxes(img,1,2)
# #             # One more dimension for the channels
# #             img = np.expand_dims(img, axis=3)

# #             X[c,:,:,:,:] = img;
# #         return X/np.max(X) #We normalize between 0 an 1

# def __data_generation(Batch_ids, Batch_Y):
#     'Generates data containing batch_size samples'
#     # Initialization
#     X = np.zeros((batch_size, *dim, n_channels))

#     # Generate data
#     for c, i in enumerate(Batch_ids): #count, element
#         case_path = os.path.join(data_path, le.inverse_transform([Batch_Y[c]])[0])
#         img_path = os.path.join(case_path, i);

#         # load nibabel Method
#         img = nib.load(img_path).get_fdata()    

#         # # One more dimension for the channels
#         img = np.expand_dims(img, axis=3)

#         X[c,:,:,:,:] = img;
#     return X/np.max(X) #We normalize between 0 an 1
        
# t1 = time.time()
# img_data = __data_generation(X,Y)
# t2 = time.time()
# print(t2-t1)
# print('segundos')
# print(img_data.shape)
# print(type(img_data))

# for img in img_data:
#   # fig, ax1 = plt.subplots(1, 1, figsize = (15,15))
#   # ax1.imshow(rotate(montage(img[24:-23,:,:,0]), -90, resize=True), cmap ='gray')
#   plt.imshow(img[:,:,72, 0], cmap='gray' )
#   plt.show()

In [12]:
# from pathlib import Path
# list(Path(project_dir).rglob("*.[nN][iI][iI].[gG][zZ]"))

In [13]:
# import os
# from glob import glob
# files = [os.path.relpath(file_dir, project_dir) for x in os.walk(project_dir) for file_dir in glob(os.path.join(x[0], '*.nii.gz'))]


In [14]:
# import os
  
# # Path
# path = 'drive/MyDrive/Colab Notebooks/TFM/DATA/Train/CN/098_S_0896_I56031_masked_basic.nii.gz'
  
# # Path of Start directory
# start = project_dir
  
# # Compute the relative file path
# # to the given path from the 
# # the given start directory.
# relative_path = os.path.relpath(path, start)
  
# # Print the relative file path
# # to the given path from the 
# # the given start directory.
# print(relative_path)


**Prepare data**

In [15]:
training_generator = DataGenerator(data_path=project_dir + '/Train/',
                                   dim=images_shape,
                                   batch_size = batch_size,
                                   n_channels = n_channels,
                                   shuffle=True)
# valid_generator = DataGenerator(val_ids)
test_generator = DataGenerator(data_path=project_dir + '/Test/',
                                   dim=images_shape,
                                   batch_size = batch_size,
                                   n_channels = n_channels,
                                   shuffle=False)


In [16]:
# print('len ', str(training_generator.__len__()))
# print(type(training_generator.__len__()))
# print('len ', str(len(training_generator)))
# # X, y = training_generator.__getitem__(1)
# X, y = training_generator[1]

# plt.imshow(X[8,:,:,72, 0], cmap='gray' )
# plt.show()

# print(type(training_generator))
# # y
# # X

**MODEL**

In [17]:
from tensorflow.keras.models import Model
sample_shape=(192,192,160,1)
# Create the model
# model = Sequential()
# model.add(Conv3D(32, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform', input_shape=sample_shape))
# model.add(MaxPooling3D(pool_size=(2, 2, 2)))
# model.add(BatchNormalization(center=True, scale=True))
# model.add(Dropout(0.5))
# # model.add(Conv3D(64, kernel_size=(3, 3, 3), activation='relu', kernel_initializer='he_uniform'))
# # model.add(MaxPooling3D(pool_size=(2, 2, 2)))
# # model.add(BatchNormalization(center=True, scale=True))
# # model.add(Dropout(0.5))
# model.add(Flatten())
# model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
# model.add(Dense(256, activation='relu', kernel_initializer='he_uniform'))
# model.add(Dense(10, activation='softmax'))

# model.summary()


# from resnet3d import Resnet3DBuilder
# base_model = Resnet3DBuilder.build_resnet_18(
#     input_shape=sample_shape,
#     num_outputs=num_classes,
#     reg_factor=0.04
# )

# # add the final layers and compile
# fc_layer = tf.keras.layers.Dense(512, activation='relu')(base_model.layers[-2].output)
# fc_layer = tf.keras.layers.Dropout(0.8)(fc_layer)
# output_layer = tf.keras.layers.Dense(N_CLASSES, activation='softmax')(fc_layer)
# model = tf.keras.models.Model(inputs=base_model.input, outputs=output_layer)
# optimizer = tf.keras.optimizers.Adam(lr=0.00001)
# model.compile(optimizer=optimizer, 
#               loss='categorical_crossentropy', 
#               metrics=['acc'])

# # train the model
# model.fit(x=image_tensor, y=label_tensor, epochs=50, 
#           steps_per_epoch=STEPS_PER_EPOCH, 
#           validation_data=val_tensor, 
#           validation_steps=VALIDATION_STEPS)


def get_model(width=192, height=192, depth=160):
    """Build a 3D convolutional neural network model."""

    inputs = keras.Input((width, height, depth, 1))

    x = Conv3D(filters=64, kernel_size=3, activation="relu")(inputs)
    x = MaxPool3D(pool_size=2)(x)
    x = BatchNormalization()(x)

    x = Conv3D(filters=64, kernel_size=3, activation="relu")(x)
    x = MaxPool3D(pool_size=2)(x)
    x = BatchNormalization()(x)

    x = Conv3D(filters=128, kernel_size=3, activation="relu")(x)
    x = MaxPool3D(pool_size=2)(x)
    x = BatchNormalization()(x)

    x = Conv3D(filters=256, kernel_size=3, activation="relu")(x)
    x = MaxPool3D(pool_size=2)(x)
    x = BatchNormalization()(x)

    x = GlobalAveragePooling3D()(x)
    x = Dense(units=512, activation="relu")(x)
    x = Dropout(0.3)(x)

    outputs = Dense(units=1, activation="sigmoid")(x)

    # Define the model.
    model = keras.Model(inputs, outputs, name="3dcnn")
    return model


# Build model.
model = get_model(width=192, height=192, depth=160)
model.summary()

Model: "3dcnn"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
input_1 (InputLayer)         [(None, 192, 192, 160, 1) 0         
_________________________________________________________________
conv3d (Conv3D)              (None, 190, 190, 158, 64) 1792      
_________________________________________________________________
max_pooling3d (MaxPooling3D) (None, 95, 95, 79, 64)    0         
_________________________________________________________________
batch_normalization (BatchNo (None, 95, 95, 79, 64)    256       
_________________________________________________________________
conv3d_1 (Conv3D)            (None, 93, 93, 77, 64)    110656    
_________________________________________________________________
max_pooling3d_1 (MaxPooling3 (None, 46, 46, 38, 64)    0         
_________________________________________________________________
batch_normalization_1 (Batch (None, 46, 46, 38, 64)    256   

In [18]:
# plot_model(model, 
#            show_shapes = True,
#            show_dtype=False,
#            show_layer_names = True, 
#            rankdir = 'TB', 
#            expand_nested = False, 
#            dpi = 70)

In [19]:
opt = Adam(0.001, decay=1e-6)

In [20]:
# Compile the model
model.compile(loss='categorical_crossentropy',
              optimizer=opt,
              metrics=['accuracy'])

In [ ]:
# Fit data to model
history = model.fit(x=training_generator,
                    # batch_size=batch_size,
                    epochs=epochs,
                    verbose=1,
                    validation_data = test_generator)

Epoch 1/100
10/30 [=========>....................] - ETA: 20:31 - loss: 1.1921e-07 - accuracy: 0.5000